<a href="https://colab.research.google.com/github/chirayu2/SentimentalAnalysis/blob/main/Chirayu_SGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape
!pip install wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.1 MB/s eta 0:00:00


In [ ]:

import snscrape.modules.twitter as sntwitter
import pandas as pd

# Created a list to append all tweet attributes(data)
tweets = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet_1 in enumerate(sntwitter.TwitterSearchScraper('Narendra Modi since:2023-04-02 until:2023-04-03').get_items()):
    if i>100:
        break

    tweets.append(tweet_1.content)

# Creating a dataframe from the tweets list above 
 #tweets_df = pd.DataFrame(attributes_container, columns=["Date Created", "Number of Likes", "Source of Tweet", "Tweets"])
tweets_df = pd.DataFrame (tweets, columns= ['Tweets'])

<ipython-input-53-e79adaaf2cdc>:12: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append(tweet_1.content)


In [ ]:
tweets_df

,Tweets
0,@DrAlkaRay2 @narendramodi @AmitShah @blsanthos...
1,@BJP4TamilNadu @narendramodi மகிழ்ச்சி
2,@KapilMishra_IND ये सब तुमने प्रधानमंत्री @nar...
3,@Swamy39 @mm_0774 @jrcdgr8 Is he a Gay or a wo...
4,समाज में हर वर्गों के साथ बिना किसी भेदभाव के ...
...,...
96,@PiyushGoyal @narendramodi https://t.co/q3LZow...
97,@BijuGopi7 @khushsundar @DrSJaishankar @narend...
98,@ChibSubhash @narendramodi @dpradhanbjp @smrit...
99,Why Sharad Pawar did not talk about modi and ...


In [ ]:
#function to clean the text
import re
import matplotlib.pyplot as pl
from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np

def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #removed @ mentions
    text = re.sub(r'#', '', text) # removed # tags
    text = re.sub(r'RT[\s]+','', text) # removed Retweets
    text = re.sub(r'https?:\/\/\S+','', text) # removed hyperlinks

    return text

tweets_df['Tweets'] = tweets_df['Tweets'].apply(clean_text)
  

In [ ]:
tweets_df

,Tweets
0,बहुत सुन्दर
1,மகிழ்ச்சி
2,_IND ये सब तुमने प्रधानमंत्री ji के लिए बोला ...
3,_0774 Is he a Gay or a womanizer?\n\nMacron ...
4,समाज में हर वर्गों के साथ बिना किसी भेदभाव के ...
...,...
96,
97,Proceed legally against this university fo...
98,
99,Why Sharad Pawar did not talk about modi and ...


In [ ]:
#load the model and tokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained (roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)



def getScores (text):
  encoded_tweet = tokenizer ( text, return_tensors = 'pt')
  output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  return scores



tweets_df['Sentiment'] = tweets_df['Tweets'].apply(getScores)

neg_t = 0
pos_t = 0
net_t = 0
for i in range(100):
  temp = tweets_df['Sentiment'][i][0]
  neg_t = neg_t + temp
  temp = tweets_df['Sentiment'][i][1]
  net_t = net_t + temp
  temp = tweets_df['Sentiment'][i][2]
  pos_t = pos_t + temp
avg_neg = neg_t/100
avg_net = net_t/100
avg_pos = pos_t/100
print('avg Positive = ', avg_pos)
print('avg Negative = ', avg_neg)
print('avg Neutral = ', avg_net)


avg Positive =  0.1287367347860709
avg Negative =  0.2677127420785837
avg Neutral =  0.6035505226999521


In [ ]:
if(avg_pos > avg_neg):
  print("On ignoring the neutral opinions, the overall sentiment is positive")
elif(avg_neg > avg_pos):
  print(('On ignoring the neutral opinions, the overall sentiment is negative'))

On ignoring the neutral opinions, the overall sentiment is negative
